# Multi-Agent Report Generation with AgentWorkflow

In this notebook, we will explore how to use the `AgentWorkflow` class to create multi-agent systems. Specifically, we will create a system that can generate a report on a given topic.

## Setup

In this example, we demonstrate the use of local serving of a `qwen3-8b` Small Language Model (SML) as our LLM, served by LM Studio, and a state-of-the-art model `gemini-2.5-flash` hosted by Google Cloud. For all supported LLM inference providers and models, check out the [examples documentation](https://docs.llamaindex.ai/en/stable/examples/llm/openai/) or [LlamaHub](https://llamahub.ai/?tab=llms) for a list of all supported LLMs and how to install/use them.

If we wanted, each agent could have a different LLM, but for this example, we will use the same LLM for all agents.

In [ ]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Environment variables for local LM studio inference
model = "qwen/qwen3-8b"
base_url = "http://127.0.0.1:1234/v1"
api_key = ""

# Environment variable for Google GenAI API inference
google_api_key = os.getenv("GOOGLE_API_KEY", "")
google_model = "gemini-2.5-flash"

In [ ]:
# Fix for "RuntimeError: This event loop is already running"
import nest_asyncio
nest_asyncio.apply()

from llama_index.llms.lmstudio import LMStudio
from llama_index.core.base.llms.types import ChatMessage, MessageRole

# Initialize the LMStudio client with the model and base URL
#llm = LMStudio(
#    model_name=model,
#    base_url=base_url,
#    temperature=0.7,
#)

from llama_index.llms.google_genai import GoogleGenAI
# Initialize the Google GenAI client with the API key
llm = GoogleGenAI(
    model=google_model,
    api_key=google_api_key,  
)

In [ ]:
# Test the LLM endpoint with a simple prompt
response = llm.complete("What is 2+2?")
print(str(response))

## System Design

Our system will have three agents:

1. A `ResearchAgent` that will search the web for information on the given topic.
2. A `WriteAgent` that will write the report using the information found by the `ResearchAgent`.
3. A `ReviewAgent` that will review the report and provide feedback.

We will use the `AgentWorkflow` class to create a multi-agent system that will execute these agents in order.

While there are many ways to implement this system, in this case, we will use a few tools to help with the research and writing processes.

1. A `web_search` tool to search the web for information on the given topic.
2. A `query_engine` tool to query local documents via Query Engine (RAG)
3. A `record_notes` tool to record notes on the given topic.
4. A `write_report` tool to write the report using the information found by the `ResearchAgent`.
5. A `review_report` tool to review the report and provide feedback.

Utilizing the `Context` class, we can pass state between agents, and each agent will have access to the current state of the system.


In [ ]:
# Build index for the query engine tool from data sources in local ./data directory
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/"
    )
    index = load_index_from_storage(storage_context)
    index_loaded = True
except:
    index_loaded = False

if not index_loaded:
    # load data
    docs = SimpleDirectoryReader(
        input_dir=["./data/"].
        file_extractors=[".txt", ".pdf"],
    ).load_data()

    # build index
    index = VectorStoreIndex.from_documents(docs)

    # persist index
    index.storage_context.persist(persist_dir="./storage/")

In [ ]:
# Define query engine tool
from llama_index.core.tools import QueryEngineTool, ToolMetadata

async def query_data(query: str) -> str:
    """Useful for querying local data on the history of the Internet"""
    engine = index.as_query_engine(similarity_top_k=3)
    return str(await engine.query(query))


In [ ]:
from tavily import AsyncTavilyClient
from llama_index.core.workflow import Context


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=os.environ.get("TAVILY_SEARCH_API_KEY"))
    return str(await client.search(query))


async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic. Your input should be notes with a title to save the notes under."""
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."


async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic. Your input should be a markdown formatted report."""
    current_state = await ctx.get("state")
    current_state["report_content"] = report_content
    await ctx.set("state", current_state)
    return "Report written."


async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback. Your input should be a review of the report."""
    current_state = await ctx.get("state")
    current_state["review"] = review
    await ctx.set("state", current_state)
    return "Report reviewed."

With our tools defined, we can now create our agents.

If the LLM you are using supports tool calling, you can use the `FunctionAgent` class. Otherwise, you can use the `ReActAgent` class.

Here, the name and description of each agent is used so that the system knows what each agent is responsible for and when to hand off control to the next agent.

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

test_agent_search = ReActAgent(
    tools=[search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

test_agent_query = ReActAgent(
    tools=[query_data],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

research_agent = ReActAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic."
        "You should first search the web with the search_web tool for information on the topic and then record notes on the topic. "
        "Once notes are recorded and once you are satisfied, you should always hand off control to the WriteAgent to write a report on the topic. "
        "You should have at least some notes on a topic before handing off control to the WriteAgent."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = ReActAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = ReActAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review the write report and provide feedback. "
        "Your review should either approve the current report or request changes for the WriteAgent to implement. "
        "If you have feedback that requires changes, you should hand off control to the WriteAgent to implement the changes after submitting the review."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

## Testing a single agent

Use the test agent to ensure that agent and tools work with the chosen model

In [ ]:
response = await test_agent_search.run(user_msg="What is the weather in San Francisco?")
print(str(response))

In [ ]:
response = await test_agent_query.run(user_msg="Who published the first paper on packet switching theory?")
print(str(response))

## Running the Workflow

With our agents defined, we can create our `AgentWorkflow` and run it.

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

As the workflow is running, we will stream the events to get an idea of what is happening under the hood.

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg=(
        "Search for the history of internet and write me a report on it. "
        "Briefly describe the history of the internet, including the development of the internet, the development of the web, "
        "and the development of the internet in the 21st century."
    )
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # if isinstance(event, AgentStream):
    #     if event.delta:
    #         print(event.delta, end="", flush=True)
    # elif isinstance(event, AgentInput):
    #     print("📥 Input:", event.input)
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

Now, we can retrieve the final report in the system for ourselves.

In [ ]:
state = await handler.ctx.store.get("state")
print(state["report_content"])